In [ ]:
!pip install transformers

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from transformers import AutoTokenizer
from transformers import GPT2LMHeadModel

In [ ]:
# 모델 및 토크나이저 로드
tokenizer = AutoTokenizer.from_pretrained('skt/kogpt2-base-v2', bos_token='</s>', eos_token='</s>', pad_token='<pad>')
model = GPT2LMHeadModel.from_pretrained('skt/kogpt2-base-v2')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.00k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.83M [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/513M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [ ]:
import pandas as pd
import tqdm

In [ ]:
# 데이터셋 로드 및 전처리
df= pd.read_csv('/content/drive/MyDrive/DBDBDeep/last_df.csv')
df.head(10)

,Question,Answer
0,면진장치가 뭐야?,면진장치란 지반에서 오는 진동 에너지를 흡수하여 건물에 주는 진동을 줄여주는 진동 ...
1,면진장치가 뭐야?,"면진장치란 건물의 지반에서 발생하는 진동 에너지를 흡수하여 건물을 보호하고, 진동을..."
2,면진장치가 뭐야?,면진장치란 지반으로부터 발생하는 진동 에너지를 흡수하여 건물에 전달되는 진동을 줄여...
3,면진장치가 뭐야?,면진장치는 건물의 지반으로부터 오는 진동 에너지를 흡수하여 건물에 전달되는 진동을 ...
4,면진장치가 뭐야?,면진장치는 건물에 오는 지반 진동의 영향을 최대한으로 흡수하여 건물에 전달되는 진동...
5,면진장치에 사용되는 주요 기술은 무엇인가요?,면진장치란 지반에서 오는 진동 에너지를 흡수하여 건물에 주는 진동을 줄여주는 진동 ...
6,면진장치에 사용되는 주요 기술은 무엇인가요?,"면진장치란 건물의 지반에서 발생하는 진동 에너지를 흡수하여 건물을 보호하고, 진동을..."
7,면진장치에 사용되는 주요 기술은 무엇인가요?,면진장치란 지반으로부터 발생하는 진동 에너지를 흡수하여 건물에 전달되는 진동을 줄여...
8,면진장치에 사용되는 주요 기술은 무엇인가요?,면진장치는 건물의 지반으로부터 오는 진동 에너지를 흡수하여 건물에 전달되는 진동을 ...
9,면진장치에 사용되는 주요 기술은 무엇인가요?,면진장치는 건물에 오는 지반 진동의 영향을 최대한으로 흡수하여 건물에 전달되는 진동...


In [ ]:
len(df)

7430

In [ ]:
batch_size = 10
train_data=df

In [ ]:
from transformers import AutoTokenizer, GPT2LMHeadModel
import pandas as pd
from torch.utils.data import Dataset, DataLoader
import tqdm
import torch

# 모델 및 토크나이저 로드
tokenizer = AutoTokenizer.from_pretrained('skt/kogpt2-base-v2', bos_token='</s>', eos_token='</s>', pad_token='<pad>')
model = GPT2LMHeadModel.from_pretrained('skt/kogpt2-base-v2')

# 데이터셋 로드 및 전처리
df = pd.read_csv('/content/drive/MyDrive/DBDBDeep/last_df.csv')
batch_size = 10

# 데이터셋 및 데이터로더 정의
class ChatDataset(Dataset):
    def __init__(self, train_data, tokenizer):
        self.train_data = train_data
        self.tokenizer = tokenizer

    def __len__(self):
        return len(self.train_data)

    def __getitem__(self, idx):
        question = self.train_data.Question.iloc[idx]
        answer = self.train_data.Answer.iloc[idx]
        bos_token = self.tokenizer.bos_token_id
        eos_token = self.tokenizer.eos_token_id

        # <usr> 사용자의 입력 <sys> 챗봇의 답변
        sent = self.tokenizer.encode('<usr>' + question + '<sys>' + answer, add_special_tokens=False)
        return torch.tensor([bos_token] + sent + [eos_token], dtype=torch.long)

def collate_fn(batch):
    return torch.nn.utils.rnn.pad_sequence(batch, batch_first=True, padding_value=tokenizer.pad_token_id)

chat_dataset = ChatDataset(df, tokenizer)
data_loader = DataLoader(chat_dataset, batch_size=batch_size, collate_fn=collate_fn)

# 모델을 GPU로 옮기기
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# 옵티마이저 설정
optimizer = torch.optim.Adam(model.parameters(), lr=3e-5, eps=1e-08)

# 초기화
EPOCHS = 50
steps = len(df) // batch_size + 1
early_stopping = True
min_delta = 0.001  # 최소 손실값 변화
patience = 3
best_loss = float('inf')
no_improvement_epochs = 1
prev_loss = float('inf')


# 주어진 에폭 수만큼 반복하여 모델을 학습
for epoch in range(EPOCHS):
    epoch_loss = 0

    for batch in tqdm.tqdm(data_loader, total=steps):
        batch = batch.to(device)
        labels = batch.clone()
        optimizer.zero_grad()
        result = model(input_ids=batch, labels=labels)
        loss = result.loss
        batch_loss = loss.mean()

        batch_loss.backward()
        optimizer.step()
        epoch_loss += batch_loss.item() / steps

    print('[Epoch: {:>4}] cost = {:>.9}'.format(epoch + 1, epoch_loss))

    # 조기 종료 확인
    if early_stopping:
        if epoch > 0:
            if prev_loss - epoch_loss < min_delta:
                no_improvement_epochs += 1
            else:
                no_improvement_epochs = 0

            if no_improvement_epochs >= patience:
                print(f'No improvement for {patience} epochs. Early stopping...')
                break

        # 이전 손실값 업데이트
        prev_loss = epoch_loss
        # 최상의 손실 갱신
        best_loss = min(best_loss, epoch_loss)


100%|█████████▉| 743/744 [01:12<00:00, 10.20it/s]


[Epoch:    1] cost = 2.3046659


100%|█████████▉| 743/744 [01:11<00:00, 10.46it/s]


[Epoch:    2] cost = 1.71223267


100%|█████████▉| 743/744 [01:11<00:00, 10.45it/s]


...

[Epoch:   32] cost = 0.125573376


100%|█████████▉| 743/744 [01:11<00:00, 10.45it/s]


[Epoch:   33] cost = 0.125272267


100%|█████████▉| 743/744 [01:11<00:00, 10.46it/s]

[Epoch:   34] cost = 0.128393365
No improvement for 3 epochs. Early stopping...


In [ ]:
# 구글 드라이브에 모델 저장
model.save_pretrained('/content/drive/MyDrive/DBDBDeep/model_skt2')
tokenizer.save_pretrained('/content/drive/MyDrive/DBDBDeep/model_skt2')

('/content/drive/MyDrive/DBDBDeep/model_skt2/tokenizer_config.json',
 '/content/drive/MyDrive/DBDBDeep/model_skt2/special_tokens_map.json',
 '/content/drive/MyDrive/DBDBDeep/model_skt2/vocab.json',
 '/content/drive/MyDrive/DBDBDeep/model_skt2/merges.txt',
 '/content/drive/MyDrive/DBDBDeep/model_skt2/added_tokens.json',
 '/content/drive/MyDrive/DBDBDeep/model_skt2/tokenizer.json')

In [ ]:
# from pytorchtools import EarlyStopping
# early_stopping = EarlyStopping(patience = 3, verbose = True)

In [ ]:
model_dir='/content/drive/MyDrive/DBDBDeep/model_skt2'

In [ ]:
# 모델 및 토크나이저 초기화
tokenizer = AutoTokenizer.from_pretrained(model_dir, bos_token='</s>', eos_token='</s>', pad_token='<pad>')
model = GPT2LMHeadModel.from_pretrained(model_dir)

In [ ]:
# 모델을 GPU로 옮기기
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)


In [ ]:
text = '벽지가 벌어졌어'

In [ ]:
sent = '<usr>' + text + '<sys>'

In [ ]:
output = model.generate(input_ids, max_length=50, early_stopping=True, eos_token_id=tokenizer.eos_token_id)

In [ ]:
input_ids = [tokenizer.bos_token_id] + tokenizer.encode(sent)
input_ids = torch.tensor([input_ids], dtype=torch.long).to(device)

In [ ]:
decoded_sentence = tokenizer.decode(output[0].tolist())

In [ ]:
output = model.generate(input_ids, max_length=50, do_sample=True, top_k=10)
tokenizer.decode(output[0].tolist())

In [ ]:
decoded_sentence.split('<sys> ')[1].replace('</s>', '')

In [ ]:
# 사용자 입력을 받아 챗봇이 응답을 반환하도록 함수 만들기

def return_answer_by_chatbot(user_text):
    sent = '<usr>' + user_text + '<sys>'
    input_ids = [tokenizer.bos_token_id] + tokenizer.encode(sent, add_special_tokens=False)   # 입력 문장을 인코딩
    input_ids = torch.tensor([input_ids], dtype=torch.long).to(device)  # 텐서로 변환하고 GPU로 이동합니다.

    # 모델을 사용하여 챗봇의 응답을 생성합. 최대 길이는 50, 샘플링을 수행하며 상위 2개 토큰을 선택.
    output = model.generate(input_ids, max_length=150, do_sample=True, top_k=2)
    sentence = tokenizer.decode(output[0].tolist())  # 토큰을 문장으로 디코딩
    chatbot_response = sentence.split('<sys> ')[1].replace('</s>', '')
    return chatbot_response

In [ ]:
user_text=input('질문을 입력하세요: ')

In [ ]:
return_answer_by_chatbot(user_text)

In [ ]:
return_answer_by_chatbot('도배할때 주의사항이 뭐야')

In [ ]:
return_answer_by_chatbot('도배 비용을 알고 싶어')

In [ ]:
return_answer_by_chatbot('벽지가 벌어졌어요')

In [ ]:
return_answer_by_chatbot('벽지 붙일때 울지 않게 하려면 어떻게 해')

In [ ]:
test = pd.read_csv('/content/drive/MyDrive/DBDBDeep/test.csv')
test.head()

In [ ]:
from tqdm import tqdm

In [ ]:
# Test 데이터셋의 모든 질의에 대한 답변으로부터 512 차원의 Embedding Vector 추출
# 평가를 위한 Embedding Vector 추출에 활용하는 모델은 'distiluse-base-multilingual-cased-v1' 이므로 반드시 확인해주세요.
from sentence_transformers import SentenceTransformer # SentenceTransformer Version 2.2.2

# Embedding Vector 추출에 활용할 모델(distiluse-base-multilingual-cased-v1) 불러오기

# 생성한 모든 응답(답변)으로부터 Embedding Vector 추출
pred_embeddings = model.encode(preds)
pred_embeddings.shape

In [ ]:
submit = pd.read_csv('/content/drive/MyDrive/DBDBDeep/sample_submission.csv')
# 제출 양식 파일(sample_submission.csv)을 활용하여 Embedding Vector로 변환한 결과를 삽입
submit.iloc[:,1:] = pred_embeddings
submit.head()

In [ ]:
# 리더보드 제출을 위한 csv파일 생성
submit.to_csv('/content/drive/MyDrive/DBDBDeep/baseline_submit.csv', index=False)